In [36]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV

# 1. 데이터 로딩
df = pd.read_csv('데이터프레임_수정_v02.csv') 

# 2. 독립변수 / 종속변수 지정
X = df[['생활 인구', '소매업', '의료기관', '교육기관', '카드소비', '서비스업', '음식점', '지역면적(km^2)', '버스정류장수', '추정 차량등록수', '근린생활시설수']]
y = df['적정 주차면수']  # = 주차면수 합 + 단속건수*가중치

In [37]:
# XGBoost GridSearch
# 1. 모델과 파라미터 정의
xgb = XGBRegressor(random_state=0)
param_grid = {
    'n_estimators': [50, 60, 75, 90, 100, 150],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.03, 0.035, 0.04, 0.045, 0.05, 0.1],
    'subsample': [0.8, 0.9, 1.0]
}

# 2. R² 기반 GridSearchCV (CV 수행)
grid_search = GridSearchCV(
    estimator=xgb,
    param_grid=param_grid,
    scoring='r2',
    cv=10,
    n_jobs=-1
)
grid_search.fit(X, y)
best_model = grid_search.best_estimator_

# 3. 교차검증 기반 성능 평가
n = X.shape[0]  # 표본 수
p = X.shape[1]  # 설명변수 수

# R²
cv_r2_rf = np.mean(cross_val_score(best_model, X, y, cv=10, scoring='r2'))

# Adjusted R²
adj_r2_rf = 1 - (1 - cv_r2_rf) * (n - 1) / (n - p - 1)

# RMSE
neg_mse_scores_rf = cross_val_score(best_model, X, y, cv=10, scoring='neg_mean_squared_error')
cv_rmse_rf = np.mean(np.sqrt(-neg_mse_scores_rf))

# MAE
neg_mae_scores_rf = cross_val_score(best_model, X, y, cv=10, scoring='neg_mean_absolute_error')
cv_mae_rf = np.mean(-neg_mae_scores_rf)

# 4. 결과 출력
print("✅ [XGBoost] 최적 하이퍼파라미터:", grid_search.best_params_)
print("📊 [XGBoost] 교차검증 기반 성능 지표:")
print(f"R²            : {cv_r2_rf:.5f}")
print(f"Adjusted R²   : {adj_r2_rf:.5f}")
print(f"RMSE          : {cv_rmse_rf:.5f}")
print(f"MAE           : {cv_mae_rf:.5f}")

c:\Users\siam2\anaconda3\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


✅ [XGBoost] 최적 하이퍼파라미터: {'learning_rate': 0.045, 'max_depth': 3, 'n_estimators': 60, 'subsample': 0.9}
📊 [XGBoost] 교차검증 기반 성능 지표:
R²            : 0.54270
Adjusted R²   : 0.38550
RMSE          : 3153.85996
MAE           : 1952.46527


In [16]:
# RandomForest GridSearch
# 1. 모델과 파라미터 정의
rf = RandomForestRegressor(random_state=0)
param_grid_rf = {
    'n_estimators': [75, 100, 150],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5]
}

# 2. GridSearchCV (R² 기준)
grid_search_rf = GridSearchCV(
    estimator=rf,
    param_grid=param_grid_rf,
    scoring='r2',
    cv=10,
    n_jobs=-1
)
grid_search_rf.fit(X, y)
best_rf = grid_search_rf.best_estimator_

# 3. 교차검증 기반 성능 평가
n = X.shape[0]  # 표본 수
p = X.shape[1]  # 설명변수 수

# R²
cv_r2_rf = np.mean(cross_val_score(best_rf, X, y, cv=10, scoring='r2'))

# Adjusted R²
adj_r2_rf = 1 - (1 - cv_r2_rf) * (n - 1) / (n - p - 1)

# RMSE
neg_mse_scores_rf = cross_val_score(best_rf, X, y, cv=10, scoring='neg_mean_squared_error')
cv_rmse_rf = np.mean(np.sqrt(-neg_mse_scores_rf))

# MAE
neg_mae_scores_rf = cross_val_score(best_rf, X, y, cv=10, scoring='neg_mean_absolute_error')
cv_mae_rf = np.mean(-neg_mae_scores_rf)

# 4. 결과 출력
print("✅ [RandomForest] 최적 하이퍼파라미터:", grid_search_rf.best_params_)
print("📊 [RandomForest] 교차검증 기반 성능 지표:")
print(f"R²            : {cv_r2_rf:.5f}")
print(f"Adjusted R²   : {adj_r2_rf:.5f}")
print(f"RMSE          : {cv_rmse_rf:.5f}")
print(f"MAE           : {cv_mae_rf:.5f}")

✅ [RandomForest] 최적 하이퍼파라미터: {'max_depth': 5, 'min_samples_split': 5, 'n_estimators': 100}
📊 [RandomForest] 교차검증 기반 성능 지표:
R²            : 0.34736
Adjusted R²   : 0.17460
RMSE          : 3161.34827
MAE           : 2067.41690


In [18]:
# Catboost GridSearch
# 1. 모델과 파라미터 정의
cat = CatBoostRegressor(verbose=0, random_state=0)
param_grid_cat = {
    'iterations': [75, 100],
    'depth': [3, 4, 5],
    'learning_rate': [0.03, 0.05]
}

# 2. GridSearchCV (R² 기준)
grid_search_cat = GridSearchCV(
    estimator=cat,
    param_grid=param_grid_cat,
    scoring='r2',
    cv=10,
    n_jobs=-1
)
grid_search_cat.fit(X, y)
best_cat = grid_search_cat.best_estimator_

# 3. 교차검증 기반 성능 평가
n = X.shape[0]  # 표본 수
p = X.shape[1]  # 설명변수 수

# R²
cv_r2_cat = np.mean(cross_val_score(best_cat, X, y, cv=10, scoring='r2'))

# Adjusted R²
adj_r2_cat = 1 - (1 - cv_r2_cat) * (n - 1) / (n - p - 1)

# RMSE
neg_mse_scores_cat = cross_val_score(best_cat, X, y, cv=10, scoring='neg_mean_squared_error')
cv_rmse_cat = np.mean(np.sqrt(-neg_mse_scores_cat))

# MAE
neg_mae_scores_cat = cross_val_score(best_cat, X, y, cv=10, scoring='neg_mean_absolute_error')
cv_mae_cat = np.mean(-neg_mae_scores_cat)

# 4. 결과 출력
print("✅ [CatBoost] 최적 하이퍼파라미터:", grid_search_cat.best_params_)
print("📊 [CatBoost] 교차검증 기반 성능 지표:")
print(f"R²            : {cv_r2_cat:.5f}")
print(f"Adjusted R²   : {adj_r2_cat:.5f}")
print(f"RMSE          : {cv_rmse_cat:.5f}")
print(f"MAE           : {cv_mae_cat:.5f}")

✅ [CatBoost] 최적 하이퍼파라미터: {'depth': 3, 'iterations': 100, 'learning_rate': 0.05}
📊 [CatBoost] 교차검증 기반 성능 지표:
R²            : 0.47940
Adjusted R²   : 0.34159
RMSE          : 3269.22198
MAE           : 2142.51526


성능이 가장 좋은 XGBoost를 사용 모델로 채택합니다.

이후 변수제거법(Backward Feature Elimination)으로 특정 변수를 제거했을 때 성능이 올라가는지 확인합니다.  
하나씩 변수를 제거해가며, **교차검증 기반 성능(R², RMSE 등)**을 비교합니다.

In [38]:
from sklearn.model_selection import cross_val_score
import numpy as np
import pandas as pd
from xgboost import XGBRegressor

# 전체 변수 목록
base_features = X.columns.tolist()

# 기준 모델
base_model = XGBRegressor(random_state=0)
base_r2 = np.mean(cross_val_score(base_model, X, y, cv=10, scoring='r2'))

results = []

for feature in base_features:
    reduced_features = [f for f in base_features if f != feature]
    X_reduced = X[reduced_features]
    
    model = XGBRegressor(random_state=0)
    r2 = np.mean(cross_val_score(model, X_reduced, y, cv=10, scoring='r2'))
    
    results.append({
        'Removed_Feature': feature,
        'CV_R2_after_removal': r2,
        'R2_Change': r2 - base_r2
    })

results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by='R2_Change', ascending=False)

print("✅ 변수 제거 시 교차검증 R² 변화:")
print(results_df)

✅ 변수 제거 시 교차검증 R² 변화:
   Removed_Feature  CV_R2_after_removal  R2_Change
6              음식점             0.539076   0.084829
4             카드소비             0.523404   0.069158
0            생활 인구             0.516029   0.061783
9         추정 차량등록수             0.462495   0.008248
3             교육기관             0.458777   0.004531
2             의료기관             0.449295  -0.004951
8           버스정류장수             0.445755  -0.008491
5             서비스업             0.422528  -0.031718
10         근린생활시설수             0.403110  -0.051137
7       지역면적(km^2)             0.285101  -0.169145
1              소매업            -0.497788  -0.952034


변수 제거시 R2값이 크게 오르는 카드소비, 생활인구, 음식점 변수를 제거하고 다시 XGBoost 진행

In [53]:
X = df[['추정 차량등록수', '교육기관', '의료기관', '버스정류장수', '서비스업', '근린생활시설수', '지역면적(km^2)', '소매업']]

# 1. 최적 파라미터 적용
xgb_model = XGBRegressor(
    learning_rate=0.05,
    max_depth=3,
    n_estimators=50,
    subsample=0.9,
    random_state=8
)

# 2. 교차검증 기반 성능 평가
n = X.shape[0]  # 표본 수
p = X.shape[1]  # 설명변수 수

# R²
cv_r2 = np.mean(cross_val_score(xgb_model, X, y, cv=10, scoring='r2'))

# Adjusted R²
adj_r2 = 1 - (1 - cv_r2) * (n - 1) / (n - p - 1)

# RMSE
neg_mse_scores = cross_val_score(xgb_model, X, y, cv=10, scoring='neg_mean_squared_error')
cv_rmse = np.mean(np.sqrt(-neg_mse_scores))

# MAE
neg_mae_scores = cross_val_score(xgb_model, X, y, cv=10, scoring='neg_mean_absolute_error')
cv_mae = np.mean(-neg_mae_scores)

# 3. 결과 출력
print("📌 [XGBoost] 지정 파라미터:")
print("{'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 50, 'subsample': 0.9}")
print("📊 교차검증 기반 성능 지표:")
print(f"R²            : {cv_r2:.5f}")
print(f"Adjusted R²   : {adj_r2:.5f}")
print(f"RMSE          : {cv_rmse:.5f}")
print(f"MAE           : {cv_mae:.5f}")

📌 [XGBoost] 지정 파라미터:
{'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 50, 'subsample': 0.9}
📊 교차검증 기반 성능 지표:
R²            : 0.56861
Adjusted R²   : 0.47001
RMSE          : 3168.06301
MAE           : 1956.15065


In [55]:
# 4. 전체 데이터로 모델 학습 (예측을 위해 필요)
xgb_model.fit(X, y)

# 5. 예측
df['예측_적정주차면수'] = xgb_model.predict(X)

# 6. 필요 주차면수 계산
df['필요주차면수'] = df['예측_적정주차면수'] - df['주차면수 합']

# 7. 우선순위 계산
df['우선순위지수'] = df['필요주차면수'] * (df['유입인구']/df['유입인구'].sum(axis = 0))
df_sorted = df.sort_values(by='우선순위지수', ascending=False)

print("\n🏆 우선 설치 대상 상위 10개 행정동:")
print(df_sorted[['행정동', '필요주차면수', '우선순위지수']].head(10))



🏆 우선 설치 대상 상위 10개 행정동:
     행정동        필요주차면수      우선순위지수
25   인계동  14628.623047  796.208926
31   행궁동  18817.853516  372.758873
37  광교1동   3041.166992  171.763615
27   매산동   5044.479004  158.323252
16  권선1동   3331.104004  119.393350
34  매탄3동   2485.748535   96.177483
36   원천동   1925.476074   89.573681
13    평동   2346.214844   83.529119
20   금곡동   2737.037598   79.646404
41  영통3동   1526.047363   57.006199
